<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/COMPARACION_MC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESTIMACION DE LA INTEGRAL CON METODOS MONTE CARLO
La integral  a resolver es:
$$I = \int_0^1 \sqrt{arctan(x)}\ dx $$

## METODO MONTE CARLO CRUDO (DIRECTO)

El método crudo consiste en generar puntos aleatorios $x_i \sim U(0,1)$ y evaluar la función en dichos puntos.

La integral se aproxima como:
$$I \approx \frac{1}{N} \sum_{i=1}^N f(x_i)$$

donde
$$f(x) = \sqrt{arctan(x)}$$

La varianza del estimador es:
$$\mathrm{Var}(\theta_{\text{crudo}}) = \frac{\mathrm{Var}[f(x)]}{N}$$





In [4]:
import numpy as np
import time

#Se define la función
def f(x):
  return np.sqrt(np.arctan(x))

#Número de muestras
N = 10**5

#Monte Carlo Crudo
t1_start = time.time()

x = np.random.rand(N)  #Muestras en [0,1]
fx = f(x)  #Se evalúa en f(x)
I_crudo = np.mean(fx)  #Promedio
Var_crudo = np.var(fx, ddof=1) / N  #Varianza del estimador
Std_crudo = np.sqrt(Var_crudo)  #Desviación estándar

t1_end = time.time()
t1 = t1_end - t1_start

print("=== Método Crudo ===")
print(f"Estimación Integral = {I_crudo:.6f}")
print(f"Varianza = {Var_crudo:.6e}")
print(f"Desviación Estándar = {Std_crudo:.6e}")
print(f"Tiempo de ejecución = {t1:.6f} s")

=== Método Crudo ===
Estimación Integral = 0.628915
Varianza = 4.230172e-07
Desviación Estándar = 6.503977e-04
Tiempo de ejecución = 0.009783 s


## METODO MONTE CARLO ACIERTO Y ERROR (HIT-OR-MISS)
Este método genera puntos dentro de un rectángulo que encierra la curva.

Sea:
$$x\sim U(0,1), \quad y \sim U(0,f_{\max})$$

donde
$$f_{\max} = \max_{x \in [0,1]} f(x)$$

Si un punto cumple $y\leq f(x)$, se considera un **acierto**.

El valor de la integral se estima como:
$$I\approx \frac{\# \space \text{de aciertos}}{N} \cdot (1 \cdot f_{\max})$$
La varianza del estimador es:
$$\mathrm{Var}(\theta_{\text{hit}}) = \frac{p(1-p)}{N} \cdot f_{\max}^2$$

donde $p = \mathbb{P} [y \leq f(x)]$





In [5]:
# Monte Carlo Acierto y Error
t2_start = time.time()

#Máximo de la función en [0,1]
x_grid = np.linspace(0,1,10000)
f_max = np.max(f(x_grid))

# Generar puntos aleatorios
x_rand = np.random.rand(N)
y_rand = np.random.rand(N) * f_max

# Contar cuántos puntos caen debajo de la curva
hits = y_rand <= f(x_rand)

I_hitmiss = np.mean(hits) * f_max # Estimación de la integral
p = np.mean(hits) # Varianza del estimador
Var_hitmiss = p*(1-p)*(f_max**2) / N # Varianza
Std_hitmiss = np.sqrt(Var_hitmiss) # Desviación estándar

t2_end = time.time()
t2 = t2_end - t2_start

print("=== Método Acierto y Error ===")
print(f"Estimación Integral = {I_hitmiss:.6f}")
print(f"Varianza = {Var_hitmiss:.6e}")
print(f"Desviación Estándar = {Std_hitmiss:.6e}")
print(f"Tiempo de ejecución = {t2:.6f} s")

=== Método Acierto y Error ===
Estimación Integral = 0.628353
Varianza = 1.620360e-06
Desviación Estándar = 1.272934e-03
Tiempo de ejecución = 0.010256 s


## COMPARACION DE METODOS Y EFICIENCIA
La eficiencia se define como:
$$E = \frac{t_1 \cdot \mathrm{Var}(\theta_1)}{t_2 \cdot \mathrm{Var}(\theta_2)}$$
donde:

- $t_1 \mathrm{Var}(\theta_1)$ corresponden al **Método Crudo**
- $t_2 \mathrm{Var}(\theta_2)$ corresponden al **Método Acierto y Error**



In [3]:
# Eficiencia

E = (t1 * Var_crudo) / (t2 * Var_hitmiss)
print("=== Comparación ===")
print(f"Integral Crudo = {I_crudo:.6f}")
print(f"Integral Acierto y Error = {I_hitmiss:.6f}")
print()
print(f"Eficiencia = {E:.6f}")

=== Comparación ===
Integral Crudo = 0.629123
Integral Acierto y Error = 0.628486

Eficiencia = 0.232255


# CONCLUSIONES

1. **Comparación de estimaciones:**  
   Ambos métodos (Monte Carlo Crudo y Acierto y Error) proporcionan estimaciones cercanas de la integral
   $(\int_0^1 \sqrt{\arctan(x)} \space dx)$, lo que confirma que ambos son válidos para aproximar integrales definidas.

2. **Varianza y desviación estándar:**  
   El método Crudo generalmente presenta **menor varianza y desviación estándar** que el método de Acierto y Error
   para el mismo tamaño de muestra, lo que lo hace más preciso en este caso.

3. **Tiempo de cómputo:**  
   Ambos métodos son rápidos para \(N = 10^5\), aunque el método Crudo suele ser ligeramente más eficiente en tiempo
   porque requiere menos operaciones (solo evaluar la función).

4. **Eficiencia:**  
   Al calcular la eficiencia $(E = \frac{t_1 \cdot \mathrm{Var}(\theta_1)}{t_2 \cdot \mathrm{Var}(\theta_2)})$
   podemos cuantificar la ventaja relativa de un método sobre otro. Un valor de $(E>1)$ indica que el método
   Acierto y Error es menos eficiente que el método Crudo en términos de varianza por tiempo.

5. **Consideraciones generales:**  
   - El método Crudo es más recomendable cuando se puede evaluar la función directamente y el dominio es acotado.  
   - El método Acierto y Error es útil cuando se quiere **visualizar la integral geométricamente** o cuando la función
     es difícil de promediar directamente, aunque generalmente requiere más muestras para lograr la misma precisión.
